In [16]:
import kagglehub
import os
from pathlib import Path
import pandas as pd

os.environ['KAGGLEHUB_CACHE'] = str(Path.cwd() / "data") # Set cache directory to ./data

data_root_dir = kagglehub.dataset_download("tongpython/cat-and-dog")
print("Path to dataset cache:", data_root_dir)
train_cat_dir = Path(data_root_dir) / "training_set" / "training_set" / "cats"
train_dog_dir = Path(data_root_dir) / "training_set" / "training_set" / "dogs"
val_cat_dir = Path(data_root_dir) / "test_set" / "test_set" / "cats"
val_dog_dir = Path(data_root_dir) / "test_set" / "test_set" / "dogs"
# Glob training files
train_cat_files = list(train_cat_dir.glob("**/*.jpg"))
train_dog_files = list(train_dog_dir.glob("**/*.jpg"))
print(f"There are {len(train_cat_files)} cat images, and {len(train_dog_files)} dog images in the training dataset")
df_train = pd.DataFrame({
    'path': list(train_cat_files) + list(train_dog_files),
    'label': ['cat'] * len(list(train_cat_files)) + ['dog'] * len(list(train_dog_files))
})
df_train.to_csv('annotations_train.csv', header=False, index=False)  

# Glob validation files
val_cat_files = list(val_cat_dir.glob("**/*.jpg"))
val_dog_files = list(val_dog_dir.glob("**/*.jpg"))
print(f"There are {len(val_cat_files)} cat images, and {len(val_dog_files)} dog images in the validation dataset")
df_val = pd.DataFrame({
    'path': list(val_cat_files) + list(val_dog_files),
    'label': ['cat'] * len(list(val_cat_files)) + ['dog'] * len(list(val_dog_files))
})
df_val.to_csv('annotations_val.csv', header=False, index=False)  

Path to dataset cache: /home/pbd0/Documents/3321/3321tpl-cat_vs_dog-mlp/data/datasets/tongpython/cat-and-dog/versions/1
There are 4000 cat images, and 4005 dog images in the training dataset
There are 1011 cat images, and 1012 dog images in the validation dataset
